# Introduction to NLP in TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences of text or speech.)

Another common term for NLP problems is sequence to sequence problems (seq2seq0).

In [1]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-04f3c7a5-a460-96dd-911d-2362892f4b92)


## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-10-25 04:22:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-25 04:22:54 (83.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we are going to be using is Kaggel's introduction to NLP dataset (text samples of Tweets as disaster or not disaster).

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-10-25 04:26:31--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.128, 74.125.23.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-10-25 04:26:32 (107 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
# Unzip data
unzip_data('nlp_getting_started.zip')

## Visualizing a text dataset

To visualize our text sample , we first have to read them in, one way to do so would be to use Python: 

But I prefer to get visual staright away.

So another way to do this is to use pandas.

In [7]:
import pandas as pd

In [8]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [13]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
# What does the test dataframe looks like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [15]:
# How many example of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [16]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [18]:
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes not higher than the total number samples
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}, (real disaster)' if target > 0 else '{not real disaster}')
  print(f'Text: \n{text}\n')
  print('-----\n')

Target: 1, (real disaster)
Text: 
Property losses from northern California wildfire nearly double http://t.co/oTfW5SEkD7 via @YahooNews

-----

{not real disaster}
Text: 
The Dress Memes Have Officially Exploded On The Internet http://t.co/iBsVy2R3PH

-----

{not real disaster}
Text: 
crushed a 6 mile run tonight. awesome

-----

{not real disaster}
Text: 
#spark #song Ultimate #preparedness library: http://t.co/VsGqoLr32g Prepare Yourself For Any Catastrophe. Over http://t.co/p7UhcB13Qx

-----

{not real disaster}
Text: 
@BenignoVito @LibertyBell1000 HILLARYMASS MURDERER.

-----



### Split data into training and validation sets

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [21]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [22]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first thing you have to do before you build a model is , to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenaziation - 
* Embedding - 

### Text Vectorization (tokenization)

In [23]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [25]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [27]:
# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabluary (automatically add <00V>)
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?,
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None, # how long do you want your sequences to be.,
                                    pad_to_max_tokens=False,
                                    ) 

In [28]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [30]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [31]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15 # max length our sequences will be (e.g. how many words from tweets does a model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [32]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [33]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [34]:
# Create a sample sentence and tokenize it
sample_sentence = 'There is a flood in my street!'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [37]:
# Choose random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f'original text:\n{random_sentence}\n\nVectorized version:')
text_vectorizer([random_sentence])

original text:
.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1065, 2675,    9,    3, 2279,   10, 1848,  313, 2933, 1280,  463,
          57,  650, 1213, 3770]])>

In [38]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f'Number of words in vocab: {len(words_in_vocab)}')
print(f'5 most common words: {top_5_words}')
print(f'5 least common words: {bottom_5_words}')

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embeddingusing an Embedding Layer

To make our embedding we are going to use Tensorflow's Embedding layers.

The parameters we care most about for our embedding layser:
* `input_dim` - the size of our vocabluary
* `output_dim` - the size of output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_lenght` - length of sequences being passes to the embedding layer

In [41]:
from tensorflow.keras import layers

In [48]:
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer='uniform',
                             input_length=max_length)

In [49]:
embedding

In [50]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f'Original text:\n{random_sentence}\n\nEmbedded version:')

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
#Obama signed up to a deal that far from making the world a safer place http://t.co/E0luGBL6pb via @upi #Iran #Nuclear #IranNuclearDeal

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04832175,  0.0433634 ,  0.00675314, ..., -0.01658847,
          0.017227  , -0.04280011],
        [-0.03976973, -0.00532814,  0.01222836, ...,  0.04747048,
         -0.03696079, -0.01189349],
        [-0.01747025, -0.01334999, -0.02374917, ..., -0.01285595,
         -0.04546529, -0.00774131],
        ...,
        [ 0.04974632,  0.01720286, -0.00011943, ..., -0.00887664,
         -0.04085437,  0.03170682],
        [ 0.02673907,  0.01253015, -0.04925605, ..., -0.00183312,
          0.03515439, -0.04340773],
        [-0.03857778,  0.00210547, -0.00129975, ...,  0.02996638,
         -0.04875037,  0.02454091]]], dtype=float32)>

In [51]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04832175,  0.0433634 ,  0.00675314, -0.04850205,  0.03051139,
         0.03255928, -0.00687518, -0.0497136 , -0.04146886,  0.0049215 ,
         0.04531108, -0.03473972,  0.02704234, -0.04394274,  0.01789809,
         0.02482171,  0.03231766, -0.01320356,  0.04391282,  0.02358023,
        -0.03470846,  0.02013281, -0.04942029, -0.04814383,  0.04776276,
         0.01822979, -0.00665404, -0.02654754, -0.00299937, -0.02546748,
         0.01625158,  0.02896588,  0.04277214,  0.00333513,  0.00849798,
        -0.035167  , -0.01234554, -0.04934223,  0.02278985, -0.00217317,
         0.01561287, -0.01093591,  0.00076374, -0.01908035,  0.03907646,
         0.00294558,  0.01123537,  0.01595766,  0.00599273,  0.04725394,
         0.02608258, -0.04266198,  0.02766034,  0.04467814,  0.04681568,
         0.03497274, -0.02861099, -0.02398185,  0.01817154, -0.04575603,
         0.040215  ,  0.04680273, -0.00419028, -0.04994619,  0.02639903,
  